<a href="https://colab.research.google.com/github/rainforest01-coder/ESAA_files/blob/main/ESAA_WEEK9_%EC%88%98%EC%83%81%EC%9E%91%EB%A6%AC%EB%B7%B0_%EC%A0%84%EB%A0%A5%EC%82%AC%EC%9A%A9%EB%9F%89%EC%98%88%EC%B8%A12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://dacon.io/competitions/official/236531/codeshare/12803

# 주제 및 데이터

##주제: 2025 전력 사용량 예측 AI경진대회

##데이터
Dataset Info.

❶10개 유형* 100개 건물의 전력소비량 데이터(1시간주기, ‘24.6.1～8.31),  
 ❷기상데이터**,  
 ❸건물개요(면적, 태양광·ESS 용량 등)

* 공공, 학교, 백화점, 병원, 아파트, 호텔 등

** 기온, 강수량, 풍속, 습도, 일조, 일사



1. train.csv

85일 분량의 데이터
train 데이터 : 100개 건물들의 2024년 06월 01일부터 2024년 08월 24일까지의 데이터
일시별 기온, 강수량, 풍속, 습도, 일조, 일사 정보 포함
전력사용량(kWh) 포함


2. building_info.csv

100개 건물 정보
건물 번호, 건물 유형, 연면적, 냉방 면적, 태양광 용량, ESS 저장 용량, PCS 용량


3. test.csv

test 데이터 : 100개 건물들의 2024년 08월 25일부터 2024년 08월 31일까지의 데이터
일시별 기온, 강수량, 풍속, 습도의 예보 정보


4. sample_submission.csv

제출을 위한 양식
100개 건물들의 2024년 08월 25일부터 2024년 08월 31일까지의 전력사용량(kWh)을 예측
num_date_time은 건물번호와 시간으로 구성된 ID
해당 ID에 맞춰 전력사용량 예측값을 answer 컬럼에 기입해야 함

---

#코드리뷰
1. 데이터 전처리
* 이상치: 시각화를 통해 전력소비량 급격히 낮아지거나 높아지는 구간 발견
  * 이상치 구간을 단일값과 연속값으로 구분하여 단일값은 보간으로 대체하고 연속구간은 드롭
* 시간 변수: 사인 코사인 변환으로 하루 24시간 사이클 피처를 모델에 전달
  * 여름 기간의 계절성 반영을 위해 summersin, summercos추가
* 공휴일 변수 추가: 공휴일의 경우 눈에띄게 전력소비량 감소하므로 변수로 추가
* 임시 휴일 처리:일부 건물에서  공휴일 전후 일부 건물 전력 사용량 현저히 낮음/ 매월 2,4번째 월요일 >>휴일 지정
* 기상정보 파생 변수: 기온 습도에 대해 1,4,7일 이동평균과 시간차 기반 변화량을 생성해 기상 조건의 추세와 변동성 반영/일단위 평균 최대 최소 및 일교차 습도차 추가/불쾌지수, 냉방도시, 체감온도 반영
* 이외 변수 추가: 건물유형, 연면적, 냉방면적, 요일 시간대별 평균 전력 소비량, 휴일 여부별 평균 전력 소비량, 휴일 여부별 표준편차, 시간대별 평균 전력 소비량, 시간대별 표준편차 등
* 타겟값인 전력사용량 분포가 치우쳐있어 로그변환


2. 모델링
* 모델: XGBoost
* loss function: Weighted MSE사용하여 전력 소비량의 과소 예측에 더 크게 패널티 주도록 설계
* 5-fold교차검증 앙상블과 seed앙상블 결합
* 앙상블: 건물별 그룹/건물 유형별 그룹/ 전력사용량 상관관계별 그룹/ 건물별(변수추가)

---
#차별점 및 배울점
* 건물 유형별로 패턴이 유사하지 않은 경우를 위해 전력사용량 상관관계별 그룹화
* 전력 과소예측의 위험성을 반영해 Weighted MSE사용하여 전력 소비량의 과소 예측에 더 크게 패널티 주도록 설계
* 공휴일 이외에 임시 공휴일이나 마트등이 자주 쉬는 날을 반영해 휴일로 추가
* 데이터 시각화를 통해 이상치를 찾고 단일 데이터의 경우 보간하고 구간인경우 드롭
* 특성을 반영하는 다양한 피처 추가